In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
df = pd.read_excel('/Users/shubhaankargupta/Desktop/FINM25000/multi_asset_etf_data.xlsx', sheet_name='excess returns')

In [3]:
df = df[df.columns[1::]]

In [4]:
print(df.head())

        BWX       DBC       EEM       EFA       HYG       IEF       IYR  \
0  0.007027  0.042120 -0.000027  0.035908  0.014763 -0.001674  0.045614   
1  0.008755  0.026909  0.063224 -0.023555  0.000752 -0.001218 -0.010607   
2  0.048760  0.045514  0.027283  0.056214  0.015932  0.018369  0.046589   
3 -0.012945 -0.051124 -0.028800 -0.021461  0.001933  0.025654  0.010733   
4  0.000230 -0.042318 -0.009027 -0.011781 -0.005378 -0.004716 -0.030733   

        PSP       QAI       SPY       TIP  
0  0.040556  0.002993  0.035147  0.007618  
1  0.016995  0.005849  0.000448  0.012231  
2  0.058627  0.018989  0.028917  0.023735  
3 -0.040965  0.000600 -0.010615  0.003259  
4 -0.042254 -0.010449 -0.016542  0.007876  


In [5]:
print(df.std())
print(df.mean())

BWX    0.023899
DBC    0.048080
EEM    0.050854
EFA    0.043562
HYG    0.021918
IEF    0.018314
IYR    0.048692
PSP    0.061594
QAI    0.014166
SPY    0.041234
TIP    0.014756
dtype: float64
BWX   -0.000643
DBC   -0.000441
EEM    0.002445
EFA    0.005148
HYG    0.003448
IEF    0.001367
IYR    0.006243
PSP    0.007713
QAI    0.001611
SPY    0.010678
TIP    0.001708
dtype: float64


In [6]:
mean = df.mean()*12
vol = df.std()*12**0.5

print(mean/vol)

BWX   -0.093202
DBC   -0.031774
EEM    0.166542
EFA    0.409372
HYG    0.544873
IEF    0.258569
IYR    0.444143
PSP    0.433804
QAI    0.393838
SPY    0.897103
TIP    0.401091
dtype: float64


In [7]:
corr_matrix = df.corr()

print(corr_matrix)

          BWX       DBC       EEM       EFA       HYG       IEF       IYR  \
BWX  1.000000  0.191116  0.621673  0.602820  0.602555  0.580891  0.552557   
DBC  0.191116  1.000000  0.511667  0.500922  0.461887 -0.300207  0.280518   
EEM  0.621673  0.511667  1.000000  0.819925  0.691167  0.026704  0.584063   
EFA  0.602820  0.500922  0.819925  1.000000  0.787191  0.042639  0.699292   
HYG  0.602555  0.461887  0.691167  0.787191  1.000000  0.187258  0.739356   
IEF  0.580891 -0.300207  0.026704  0.042639  0.187258  1.000000  0.316532   
IYR  0.552557  0.280518  0.584063  0.699292  0.739356  0.316532  1.000000   
PSP  0.526692  0.453303  0.750109  0.895320  0.812157  0.022436  0.749836   
QAI  0.630276  0.475311  0.774697  0.847864  0.807893  0.179761  0.718529   
SPY  0.439994  0.432162  0.687751  0.845863  0.793518  0.000815  0.754711   
TIP  0.675151  0.109006  0.378792  0.394821  0.538648  0.754102  0.598742   

          PSP       QAI       SPY       TIP  
BWX  0.526692  0.630276  0.43

In [169]:
def tan_portfolio(mean_rets, cov_matrix):
    """
    Function to calculate tangency portfolio weights. Comes from the
    formula seen in class (Week 1).

    Args:
        mean_rets: Vector of mean returns.
        cov_matrix: Covariance matrix of returns.

    Returns:
        Vector of tangency portfolio weights.
    """
    inv_cov = np.linalg.inv(cov_matrix)
    ones = np.ones(mean_rets.shape)
    return (inv_cov @ mean_rets) / (ones.T @ inv_cov @ mean_rets)

def gmv_portfolio(cov_matrix):
    """
    Function to calculate the weights of the global minimum variance portfolio.

    Args:
        cov_matrix : Covariance matrix of returns.

    Returns:
        Vector of GMV portfolio weights.
    """
    try:
        cov_inv = np.linalg.inv(cov_matrix)
    except TypeError:
        cov_inv = np.linalg.inv(np.array(cov_matrix))

    one_vector = np.ones(len(cov_matrix.index))
    return cov_inv @ one_vector / (one_vector @ cov_inv @ (one_vector))


def mv_portfolio(mean_rets, cov_matrix, target=None, excess=True):
    """
    Function to calculate the weights of the mean-variance portfolio. If
    target is not specified, then the function will return the tangency portfolio.
    If target is specified, then we return the MV-efficient portfolio with the target
    return.
    
    **This assumes we have access to a risk-free asset**

    Args:
        mean_rets : Vector of mean returns.
        cov_matrix : Covariance matrix of returns.
        target (optional):  Target mean return. Defaults to None. Note: must be adjusted for
                            annualization the same time-frequency as the mean returns. If the
                            mean returns are monthly, the target must be monthly as well.
        excess (options): Whether or not we are using excess returns.

    Returns:
        Vector of MV portfolio weights.
    """
    w_tan = tan_portfolio(mean_rets, cov_matrix)

    if target is None:
        return w_tan
    
    elif not excess:
        w_gmv = gmv_portfolio(cov_matrix)
        delta = (target - mean_rets @ w_gmv) / (mean_rets @ w_tan - mean_rets @ w_gmv)
        return delta * w_tan + (1 - delta) * w_gmv
    else:
        ones = np.ones(mean_rets.shape)
        cov_inv = np.linalg.inv(cov_matrix)
        delta = (ones @ cov_inv @ mean_rets) / (mean_rets.T @ cov_inv @ ones) 
        return target * delta * w_tan


# Note: we are NOT annualizing here.
w_tan = mv_portfolio(df.mean(), df.cov())

w_tan_df = pd.DataFrame(w_tan, index=df.columns, columns=["Tangency Portfolio"])

# Here, we use the display() function to show both the weights and the performance summary,
# without having to make two separate cells, and not mess up the formatting caused by print().
display(
    w_tan_df.sort_values(by="Tangency Portfolio", ascending=False).style.format(
        "{:.4f}"
    )
)


,Tangency Portfolio
SPY,10.7298
IEF,9.0106
HYG,2.6349
TIP,1.3306
EEM,0.8536
EFA,0.3853
DBC,-0.1123
PSP,-1.7168
IYR,-2.3821
BWX,-6.1174


In [180]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

# List of 5 stock tickers
stock_tickers = ['TGS', 'YPF', 'BNO', 'MELI', 'SPY', 'ARGT', 'GGAL', 'SHEL', 'BP']

# Calculate the date 6 months ago from today
end_date = datetime.today()
start_date = end_date - timedelta(days=360)  # Approximate 6 months

# Download historical data for the last 6 months
stock_data = yf.download(
    stock_tickers,
    start=start_date.strftime('%Y-%m-%d'),
    end=end_date.strftime('%Y-%m-%d')
)['Close']

# Reset index to have Date as a column
stock_data = stock_data.reset_index()

# Display the first few rows
stock_data = stock_data[stock_data.columns[1::]]

print(stock_data.head())

print(stock_data.corr())

/var/folders/0f/l_wv06ws6k3063l3r6mv6xb80000gn/T/ipykernel_91862/736090658.py:13: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_data = yf.download(
[*********************100%***********************]  9 of 9 completed

Ticker       ARGT        BNO         BP       GGAL         MELI       SHEL  \
0       56.156815  32.459999  34.406853  27.165161  1683.750000  69.776009   
1       56.887924  32.119999  32.759464  27.450809  1722.050049  69.641685   
2       57.381912  32.340000  32.853603  28.098280  1703.550049  69.977516   
3       57.915421  32.509998  32.994812  27.612679  1708.650024  70.015892   
4       58.192059  32.230000  33.126602  27.288944  1761.219971  70.121445   

Ticker         SPY        TGS        YPF  
0       548.449036  18.309999  20.549999  
1       548.982300  18.160000  20.730000  
2       554.414734  18.639999  21.049999  
3       549.634277  18.700001  21.129999  
4       553.101013  18.790001  21.290001  
Ticker      ARGT       BNO        BP      GGAL      MELI      SHEL       SPY  \
Ticker                                                                         
ARGT    1.000000 -0.242743 -0.348241  0.930456  0.526266 -0.269455  0.638135   
BNO    -0.242743  1.000000  0.631

MISC: Tangency portfolio of stock tickers, was just playing around.

In [173]:
def tan_portfolio(mean_rets, cov_matrix):
    """
    Function to calculate tangency portfolio weights. Comes from the
    formula seen in class (Week 1).

    Args:
        mean_rets: Vector of mean returns.
        cov_matrix: Covariance matrix of returns.

    Returns:
        Vector of tangency portfolio weights.
    """
    inv_cov = np.linalg.inv(cov_matrix)
    ones = np.ones(mean_rets.shape)
    return (inv_cov @ mean_rets) / (ones.T @ inv_cov @ mean_rets)

def gmv_portfolio(cov_matrix):
    """
    Function to calculate the weights of the global minimum variance portfolio.

    Args:
        cov_matrix : Covariance matrix of returns.

    Returns:
        Vector of GMV portfolio weights.
    """
    try:
        cov_inv = np.linalg.inv(cov_matrix)
    except TypeError:
        cov_inv = np.linalg.inv(np.array(cov_matrix))

    one_vector = np.ones(len(cov_matrix.index))
    return cov_inv @ one_vector / (one_vector @ cov_inv @ (one_vector))


def mv_portfolio(mean_rets, cov_matrix, target=None, excess=True):
    """
    Function to calculate the weights of the mean-variance portfolio. If
    target is not specified, then the function will return the tangency portfolio.
    If target is specified, then we return the MV-efficient portfolio with the target
    return.
    
    **This assumes we have access to a risk-free asset**

    Args:
        mean_rets : Vector of mean returns.
        cov_matrix : Covariance matrix of returns.
        target (optional):  Target mean return. Defaults to None. Note: must be adjusted for
                            annualization the same time-frequency as the mean returns. If the
                            mean returns are monthly, the target must be monthly as well.
        excess (options): Whether or not we are using excess returns.

    Returns:
        Vector of MV portfolio weights.
    """
    w_tan = tan_portfolio(mean_rets, cov_matrix)

    if target is None:
        return w_tan
    
    elif not excess:
        w_gmv = gmv_portfolio(cov_matrix)
        delta = (target - mean_rets @ w_gmv) / (mean_rets @ w_tan - mean_rets @ w_gmv)
        return delta * w_tan + (1 - delta) * w_gmv
    else:
        ones = np.ones(mean_rets.shape)
        cov_inv = np.linalg.inv(cov_matrix)
        delta = (ones @ cov_inv @ mean_rets) / (mean_rets.T @ cov_inv @ ones) 
        return target * delta * w_tan

df = stock_data
# Note: we are NOT annualizing here.
w_tan = mv_portfolio(df.mean(), df.cov())

w_tan_df = pd.DataFrame(w_tan, index=df.columns, columns=["Tangency Portfolio"])

# Here, we use the display() function to show both the weights and the performance summary,
# without having to make two separate cells, and not mess up the formatting caused by print().
display(
    w_tan_df.sort_values(by="Tangency Portfolio", ascending=False).style.format(
        "{:.4f}"
    )
)


,Tangency Portfolio
Ticker,
TGS,3.8366
YPF,1.4166
MELI,-0.1011
USO,-4.1521
